<a href="https://colab.research.google.com/github/davidnoone/PHYS332_FluidExamples/blob/main/Convection_FV_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thermal (Rayleigh-Benard) convection in 2d

We wish to consider a problem of thermal convection using the non-divergent euler equations in 2d. We can formulate the problem terms of vorticity, and uses simple numerical methods to do so.

For an nearly incompressible invisid fluid that is near hydrostatic (i.e., like water, or air), the governing equations are:

$$
\frac{d u}{d t} = -\frac{\partial \Phi}{\partial x}
$$
$$
\frac{d v}{d t} = -\frac{\partial \Phi}{\partial y} + b
$$

Where the buoyancy, b, is defined

$$
b = g \frac{\theta}{\theta_0}
$$

Being related to entropy via the potential temperature, $\theta$, b is conserved, but may have an external source (e.g., such as heating)
$$
\frac{d b}{d t} = Q
$$


With incompressibility, density is assumed constant, and mass continuity is just a statement that the divergence is zero.
$$
  \frac{\partial u}{dx} + \frac{\partial v}{dy} = 0
$$

This immediately allows us to define a streamfunction as the only quantity needed to describe the velocity field. That is:

$$
    u = - \frac{\partial \Psi}{\partial y}
$$

$$
    v = \frac{\partial \Psi}{\partial x}
$$

The vorticity is related to the streamfunction as:

$$
  \zeta = \frac{\partial^2 \psi}{\partial x^2}
        + \frac{\partial^2 \psi}{\partial y^2}
$$


The momentum can be described entirely by predicting the evolution of vorticity. Taking the curl of the u and v equations we find the pressure terms cancell leading to a rearkably elegant prognostic equation for voticity:
$$
\frac{d \zeta}{d t} =
  \frac{\partial}{\partial x}(\frac{d v}{d t})
  - \frac{\partial}{\partial y}(\frac{d u}{d t})
$$

We may note that the total derivative can be expanded to find the advective terms for each quantity "A"
$$
 \frac{d A}{dt} = \frac{\partial A}{\partial t} +
    u \frac{\partial A}{\partial x} + v \frac{\partial A}{\partial y}
$$

Here "y" is considered to be the vertical, such that gravity points downward in the y coordinate. Since the flow is incompressible, the $\rho$ is constant, and this second form becomes particularly convinient for numerical methods.

With the non-divergent condition, the advection term can be written in a conservative form:

$$
  u \frac{\partial A}{\partial x} + v \frac{\partial A}{\partial y} =
  \frac{\partial (uA)}{\partial x} + \frac{\partial (vA)}{\partial y}   
$$

The final Eulerian prognostic model is therefore described with two  equations that describe conservation with the the addition  of buoyancy which can induce rotation:


$$
\frac{\partial \zeta}{\partial t} =
                      - \frac{\partial (u \zeta)}{\partial x}
                      - \frac{\partial (v \zeta)}{\partial y}
                      + \frac{\partial     b}{\partial x}
$$
And
$$
\frac{\partial b}{\partial t} =
                      - \frac{\partial (ub)}{\partial x}
                      - \frac{\partial (vb)}{\partial y} + Q
$$



## Solution scheme
From the above, it should be clear that the eevolution of this system can be developed as follows:

  1. Begining with with the $\zeta^n$ and $b^n$ at some initial time ($\zeta$ could be calculated if u and v are known, of course)
  2. Solve the poisson equation to evaluate the streamfunction from the known vorticity. i.e., obtain $\psi^n$
  3. Caluclate the velocities $(u,n)^n$ as the gradients in $\psi^n$
  4. Use the velocities to calculate the advection of $\zeta$ and b.
  5. Compute the x-gradient of b to add contributions to the acceleration due to buoyancy. At this stage a (viscous) diffusion term can also be added. (Not shown algebraically)
  6. Finally, advance $\zeta$ and b in time: step from time $n$ to $n+1$

This scheme can be repeated until the desired integration in time is complete.


# Solution method

Finite difference methods can be used to evaluate gradients. The poisson equation is solved using an iterative method that applies the finite difference analog of the Laplacian in 2d.  If advection is evaluated using finite difference methods, a substantial amount of diffusion is needed to stabilize the numerical result. This is possible, however, a more accurate scheme can also be used with better results.The stepping in time is done using a 3rd order scheme, rather than a simple single forward step, because it has some nice stability properties. This is accomplished using the weighted result from three applications of a simple forward Euler time step.



#Task
1. Run the default experiment.

1. Run the experiment again with a smaller initial bubble. Describe how the simulation is different or similar. [Look to change the "width" of the bubble"]

1. Run the experiment again, but include a diffusivity. How does the addition of diffusivity change the Courant number.



In [ ]:
from matplotlib import rc
rc('animation', html='jshtml')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from numba import njit, prange

print("Modules imported.")

Some run time initialization

In [ ]:
# Some constants
gravity = 9.81    # acceleration due to gravity
theta0  = 300.    # reference potential temperature [Kelvin]

# dimensions
nx = 40          # must be even
ny = 51          # even or odd

dx = 1.0        # grid spacing [metres]
dtime = 0.5     # time step [seconds]

#nelapse = 50    # number of steps to run
nelapse = 200    # number of steps to run
nskip = 5       # steps between plotting output
nstep = 0

xmax = nx*dx
ymax = ny*dx
xmid = np.linspace(0,xmax,nx)
ymid = np.linspace(0,ymax,ny)

# Boundary conditions: top and bottom streamfunction, and buoyancy flux
psi_top = 0.0
psi_bot = 0.0

bpert = 0.001*gravity/theta0   # Random buoyancy perturbation b = g * theta/theta0
bflux = 0*0.010*gravity/theta0   # units "per second"            # {TRY THIS}
bbubl =   1.000*gravity/theta0 # add a bubble                  # {TRY THIS}

kdiff = 0.*dx*dx/50.            # [Optional] diffusivity m2/sec (little bit for stability)


# Define the working arrays here to make them global (mostly so we can plot them)
vort = np.zeros((ny,nx))
buoy = np.zeros((ny,nx)) + bpert*np.random.randn(ny,nx)
psi  = np.zeros((ny,nx))
uwnd = np.zeros((ny,nx))
vwnd = np.zeros((ny,nx))

# Initialize a hot bubble
def add_bubble(xpos, ypos, width):
  xx,yy = np.meshgrid(xmid,ymid)
  dist = np.sqrt((xx-xpos)**2 + (yy-ypos)**2)/width
  bone = np.exp(-dist**2)     # gaussian
  return bone

width = 4.0
bone = add_bubble(xmax/2.,ymax/3,width)
buoy = buoy + bone*bbubl

print('Initialization complete.')

A collection of finite difference opperators needed later on. Use the numpy "roll" function to impliment periodic boundary conditions, but explictly use one-sided derivatives in the "y" direction at boundary edges.

In [ ]:
# Centered finite difference X derivative (2nd order), with periodic BCs: dF/dx
def fd_dfdx(fld,dx):
    dfdx = 0.5*(np.roll(fld,-1, axis=1) - np.roll(fld,+1, axis=1))/dx
    return dfdx

#Centered finite difference in Z with single sided ends: dF/dy
def fd_dfdy(fld,dx):
    global ny
    dfdy = np.zeros_like(fld)
    dfdy[0     ,:] = (fld[1     ,:] - fld[     0,:])/dx
    dfdy[1:ny-1,:] = (fld[2:ny-0,:] - fld[0:ny-2,:])/(2*dx)
    dfdy[  ny-1,:] = (fld[  ny-1,:] - fld[  ny-2,:])/dx
    return dfdy

# Laplacian: del^2(F)
def fd_lap(fld,dx):
    flap =  -4*fld + np.roll(fld,-1, axis=1) + np.roll(fld,+1, axis=1)
    flap[0     ,:] = flap[0     ,:] + (fld[1     ,:]                )
    flap[1:ny-1,:] = flap[1:ny-1,:] + (fld[2:ny-0,:] + fld[0:ny-2,:])
    flap[  ny-1,:] = flap[  ny-1,:] + (                fld[  ny-2,:])
    return flap

# Curl/vorticity: del x (u,v) [Same as fd_div(v,-u)]
def fd_curl(u, v, dx):
    return fd_dfdx(v,dx) - fd_dfdy(u,dx)

# Divergence: del.(u,v)
def fd_div(u, v, dx):
    return fd_dfdx(u,dx) + fd_dfdy(v,dx)

# Advection opperator using finite differences [not very accurate]
def advect(fld,u,v,dx):
    return -1.0*(u*fd_dfdx(fld,dx) + v*fd_dfdy(fld,dx))


## Advection using finite volume methods

A finite volume method is used to evaluate the (non-linear) advection. Finite volume methods have some appealing mass conservation properties, and they can be configured to help prevent noisy solutions that come about from low order approximations to the continuous equations. This is especially important for managing non-linear terms.

Here we use a second order scheme where the advection tendency is computed as

$$
\left( \frac{\partial \zeta}{\partial t}\right)_n =
  \frac{1}{\Delta x} \left( F_{i+1/2} - F_{i-1/2} \right)
$$

The fluxes $F$ are at the interfaces (locations at "half" levles), and which must be estimated by linear reconstruction. The details of the estimation gives rise to a robust and accurate scheme. Note that an estimate of the value at each interface is can be obtained from either side. That is, the left side of the interface at $i-1/2$ can be estimated using values from the left or from the right side. Consider the two options:

$$
F_{L,i-1/2} = F_{i-1} + \frac{1}{2}\Delta F_{1-1/2}
$$
$$
F_{R,i-1/2} = F_{i} - \frac{1}{2}\Delta F_{1+1/2}
$$

The half here emerges from wanting to extrapolate half a grid length, $\Delta x$, to get from the cell center to the cell interfaces.
The gradients $\Delta F$ can adjusted to ensure monotonicity (and piece wise linear), which equivieltly ensures that no new oscillations are produced in the solution. The slope is set to be no larger (in magnitude) than estimates of $\Delta F$ from the upstream, downstream and centered pairs of values of $\zeta$ that can from the possible combinations in the range from $i-1$ to $i+1$. (_There are a range of specific limiting functions, the "minmod" method used here is simple to impliment and works well._)


Finally, the flux is estimated as the mean of these two estimates. The mean takes a special form involving a correction associate with the maximum possible (wave) speed supported by the equations of motion. This correction term effectivivly stabilizes the solution.

$$
  F = \frac{1}{2} \left[ (F_L + F_R) + \alpha (\zeta_R - \zeta_L) \right]
$$

where $\alpha = max(u_L, u_R)$. More formally, $\alpha$ is the Jacobian of the flux $\partial F/\partial u$. If we were to select $\alpha = 0$, we would identically recover the a centered finite difference method!

Using these fluxes for each interface the advection tendency is obtained.




In [ ]:
# Two-D advection using a 2nd order FV method
R = -1   # right
L = 1    # left

# Piecewise linear reconstruction of interface values, using a minmod slope limiter
def reconstruct(f, dx, axis):
    # Compute limited slope/gradient
    f_dx = 0.5*( np.roll(f,R,axis=axis) - np.roll(f,L,axis=axis) )   # centered difference
    f_dx = np.maximum(0., np.minimum(1., ( (f-np.roll(f,L,axis=axis)))/(f_dx + 1.0e-8*(f_dx==0)))) * f_dx
    f_dx = np.maximum(0., np.minimum(1., (-(f-np.roll(f,R,axis=axis)))/(f_dx + 1.0e-8*(f_dx==0)))) * f_dx

    # get left and right side estimates of the fluxes
    f_R = f + 0.5*f_dx
    f_L = f - 0.5*f_dx
    f_L = np.roll(f_L,R,axis=axis)
    return f_L, f_R

# Form the fluxes: Lax-Freidrichs type scheme
def advect_flux_1d(f, u, dx, axis):
    u_L, u_R = reconstruct(u, dx, axis)
    f_L, f_R = reconstruct(f, dx, axis)
    alpha = np.maximum( np.abs(u_L), np.abs(u_R) )
    flx = 0.5*((u_L*f_L + u_R*f_R) - alpha*(f_L - f_R))
    return flx

# Evaluate advective tendency as divegence of (interface) fluxes
def advect_tend(f,u,v,dx):
    dy = dx
    flx_x = advect_flux_1d(f, u, dx, 1)
    flx_y = advect_flux_1d(f, v, dy, 0)

    fadv = (np.roll(flx_x,L,axis=1) - flx_x)/dx + \
           (np.roll(flx_y,L,axis=0) - flx_y)/dy

    return fadv


## Inverting $\zeta$: a Poisson equation

Finding $\psi$ from $\zeta$ requires solution of the poisson equation

$$
  \nabla^2 \psi = \zeta
$$

Accomplishing this is one of the main numerical tasks in this calculation. Solving this equation effectly solves for mass continuity by defining a streamfunction.

This version uses a "hybrid" method that combines a spectral/Fourier transform in the periodic x direction, then solves a resulting 1d (in y) finite difference equation to obtain the Fourier coeffieicnts for the solution. An inverse transform is performed to get the final result on a grid. This method is a direct solution, and consequently quite fast!

A Fourier transform converts grid point representation of some field $f$ to a series of harmonics. This is an exact representation, and is a linear transformation.

$$
f(x,y) \rightarrow FFT_x \rightarrow \hat{f}(k,y)
$$

We seek a solution for Fourier coefficients $\hat{\psi}(k,y)$ with wave number k. The RHS of the Poisson equation (i.e., vorticity) is transformed using an FFT function to give $\hat{\zeta}(k,y)$.

The second derivative in of a Fourier coefficient is simply $-k^2$. That is,
$$
\frac{\partial^2 \hat{\psi}_{k}}{\partial x^2} = -k^2 \hat{\psi}_{k}
$$

And we can write for a single coefficient,

$$
  \nabla^2 \psi_k = -k^2 \hat{\psi}_{k,j}+ \frac{
         \hat{\psi}_{k,j-1} - 2\hat{\psi}_{k,j} + \hat{\psi}_{k,j+1}}
            {\Delta y} = \zeta_{k,j}
$$

Where the x derivative has the (exact!) algebraic solution.
It should be clear that this expression can be represented by a tridiagonal matrix, for which there is an efficient solver.

The resulting coefficients are tranformed back to obatin $\psi(x,y), as needed.

In this case, the boundary conditions are zero to be zero at the top and bottom, which is particularly convenient.



In [ ]:
from scipy.fft import fft, ifft, fftfreq
from scipy.linalg import solve_banded
def solve_poisson_fft_fd(dx, dy, b):
    """
    Solves the 2D Poisson equation ∇²φ = b on a domain with periodic BCs in x
    and Dirichlet BCs (φ=0) at y=0 and y=Ly using a hybrid FFT-FD method.

    Parameters:
        dx, dy : float
            Grid spacing in x and y directions.
        b : 2D np.ndarray (ny, nx)
            Source term (RHS).

    Returns:
        phi : 2D np.ndarray (ny, nx)
            Solution to the Poisson equation.
    """
    ny, nx = b.shape
    Lx = dx * nx

    # FFT in x-direction
    b_hat = fft(b, axis=1)
    kx = 2 * np.pi * fftfreq(nx, d=dx)
    kxsq = kx**2

    alpha = 1.0 / dy**2
    beta = -2.0 * alpha

    phi_hat = np.zeros_like(b_hat, dtype=complex)

    for k in range(nx):
        del2x = kxsq[k]
        rhs = b_hat[:, k].copy()

        # Tridiagonal matrix setup
        ab = np.zeros((3, ny), dtype=complex)
        ab[0, 1:] = alpha       # upper diagonal
        ab[1, :] = beta - del2x # main diagonal
        ab[2, :-1] = alpha      # lower diagonal

        # Enforce Dirichlet BCs
        rhs[0] = 0.0
        ab[1, 0] = 1.0
        ab[0, 1] = 0.0

        rhs[-1] = 0.0
        ab[1, -1] = 1.0
        ab[2, -2] = 0.0

        phi_hat[:, k] = solve_banded((1, 1), ab, rhs)

    phi = np.real(ifft(phi_hat, axis=1))
    return phi



# Integrating the convection equations
The hard part is more or less done. We now just need to set up the main formation of the time derivatives ("tendencies"), following the numbered stratergy given above. Then, finaly, do the time stepping.



In [ ]:
def tendency():
    global vort, buoy, uwnd, vwnd, psi, dx, psi_bot, psi_top
    global dtime, nstep, nskip

    #psi = psolve(psi_bot,psi_top,psi,vort,dx)
    psi = solve_poisson_fft_fd(dx, dx, vort)

    uwnd = -fd_dfdy(psi,dx)
    vwnd =  fd_dfdx(psi,dx)

    ztend = advect_tend(vort,uwnd,vwnd,dx) + kdiff*fd_lap(vort,dx) + fd_dfdx(buoy,dx)
    btend = advect_tend(buoy,uwnd,vwnd,dx) + kdiff*fd_lap(buoy,dx)

    # Heating at bottom
    btend[   0,:] += bflux

    # Cooling at top
#    btend[ny-1,:] -= bflux

    # Cooling everyhere ("atmosphere")
    btend[:,:] -= bflux/ny


    return (ztend, btend)

# Use a 3rd order SSP RK scheme to integrate: F(n+1) = F(n) + dt*dF/dt
def advance(dtime):
    global vort, buoy
    vort0 = vort
    buoy0 = buoy

    (ztend, btend) = tendency()
    vort = vort0 + dtime*ztend
    buoy = buoy0 + dtime*btend

    (ztend, btend) = tendency()
    vort = (vort0 + vort + dtime*ztend)/2.0
    buoy = (buoy0 + buoy + dtime*btend)/2.0

    (ztend, btend) = tendency()
    vort = (vort0 + 2.0*(vort + dtime*ztend))/3.0
    buoy = (buoy0 + 2.0*(buoy + dtime*btend))/3.0

    return


In [ ]:
#for nstep in range(ntime):
#    advance(dtime)

We will want to visualize the model state, define a function to plot the vorticity, buoyancy, and velocty field.

In [ ]:
def init_frame():
    cf.set_array([])
    cl.set_array([])
    return cl,cf,

def advance_frame(w):
    global vort, buoy, uwnd, vwnd, psi
    global dtime, nstep, nskip
    global cl, cf

    cno = np.max(np.sqrt(uwnd*uwnd + vwnd*vwnd))*dtime/(1.41*dx)

    # INTEGRATE THE EQUATIONS IN TIME: update to the next save time
    for n in range(nskip):
        nstep = nstep + 1
        if (cno > 2):
            print('CFL instability detected: abort! cno=',cno)
        advance(dtime)

    # Do the plot on the predefine plotting axes
    print('Frame: ',w,' nstep =',nstep, 'CFL=',cno)
    ax.set(title="Vorticity and streamfunction nstep="+str(nstep).zfill(4))
    while ax.collections:
        ax.collections[-1].remove()
    cf = ax.contourf(xx,yy,buoy,cmap=colormap,levels=19)
    cl = ax.contour(xx,yy,psi,colors='black')

    return cl, cf,



To actually run the integration,  plot at various the needed time interval.

In [ ]:
# Create an animation object, with frames created as the model runs
xx, yy = np.meshgrid(xmid,ymid)
colormap = 'jet'

fig = plt.figure(figsize=(6,6))
ax = plt.axes(xlim=(0, xmax), ylim=(0, ymax))
ax.set_aspect('equal')
ax.set(xlabel='X position [m]', ylabel='Y position [m]')
cf = ax.contourf(xx,yy,buoy,cmap=colormap,levels=19)
cl = ax.contour(xx,yy,psi,colors='black')

anim = animation.FuncAnimation(fig, advance_frame, init_func=init_frame, frames=nelapse,
                                          interval=50,repeat=True)
plt.close()

print('Finished')






In [ ]:
# Run the model by instantiating the animation.
anim

In [ ]:
# playground


